In [2]:
import pandas as pd
import numpy as np

In [3]:
import ast

In [48]:
games = pd.read_csv("steam_games.csv")
items = pd.read_csv("game_items.csv")

In [49]:
columns1 = ['genres', 'release_date', 'id']
columns = ['item_id', 'playtime_forever']


In [50]:
df_generos = pd.read_csv("steam_games.csv", usecols=columns1)
df_tiempo = pd.read_csv("game_items.csv", usecols=columns)

In [69]:
tiempo = pd.DataFrame(df_tiempo.groupby('item_id')['playtime_forever'].agg('sum')).reset_index()


In [70]:
df_generos.rename(columns={'id':'item_id'},inplace=True)
# Hacer merge (left join) con el df de user_items
df_merge = tiempo.merge(df_generos,on='item_id',how='left')
# Filtrar por los juegos que tengan playtime mayor a cero 0
df_merge = df_merge[df_merge['playtime_forever'] != 0]
# Eliminar nulos de la columna género
df_merge = df_merge.dropna(subset=['genres'])

In [71]:
# Función para convertir a lista la columna de género
def tolist(lst):
    if isinstance(lst, str) :
        lst = lst.replace("0's",'0s')
        lst =  ast.literal_eval(lst)
        # try:
        #     lst =  ast.literal_eval(lst)
        # except (SyntaxError, ValueError):
        #     return lst.lower()

    return list(lst)

In [72]:
#aplicar la funcion a df_merge para vovler una lista de la columna genero
df_merge['genres'] = df_merge['genres'].apply(lambda x : tolist(x))


In [55]:
df_merge


,item_id,playtime_forever,genres,release_date
0,10,17386015,[Action],2000-11-01
1,20,961702,[Action],1999-04-01
2,30,758991,[Action],2003-05-01
3,40,154486,[Action],2001-06-01
4,50,734562,[Action],1999-11-01
...,...,...,...,...
10967,527510,186,"[Casual, Indie, Simulation, Strategy]",2016-09-16
10970,527810,2,"[Action, Indie]",2016-09-08
10971,527890,1,"[Casual, Simulation]",2016-09-12
10972,527900,44,"[Adventure, Casual]",2016-09-14


In [73]:
df_merge['release_date'] = pd.to_datetime(df_merge['release_date'], errors='coerce')

# Identifica las filas con fechas que no se pudieron convertir.
fechas_invalidas = df_merge['release_date'].isna()

# Modifica las fechas invalidas (mes y año) para tomar el primer día del mes.
df_merge.loc[fechas_invalidas, 'release_date'] = df_merge[fechas_invalidas]['release_date'].apply(lambda x: x.replace(day=1))

In [58]:
df_merge['release_date'].info()

<class 'pandas.core.series.Series'>
Index: 8104 entries, 0 to 10974
Series name: release_date
Non-Null Count  Dtype         
--------------  -----         
8074 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 126.6 KB


In [74]:
df = df_merge.explode('genres')

# Agrupa los datos por año ('release_date') y género ('genres') y suma las horas jugadas.
grouped = df.groupby(['release_date', 'genres'])['playtime_forever'].sum().reset_index()

# Modifica la agregación para sumar las horas de juego en lugar de eliminar duplicados.
grouped = grouped.groupby(['release_date', 'genres'])['playtime_forever'].agg('sum').reset_index()

In [60]:
grouped

,release_date,genres,playtime_forever
0,1983-06-19,Action,3582
1,1983-06-19,Adventure,3582
2,1983-06-19,Casual,3582
3,1984-04-29,Action,384
4,1984-04-29,Adventure,384
...,...,...,...
10234,2018-12-01,Action,21
10235,2018-12-01,Adventure,21
10236,2018-12-01,Early Access,21
10237,2018-12-01,Indie,21


In [75]:
grouped['release_date'] = pd.to_datetime(grouped['release_date'])

# Luego, crea una nueva columna 'year' para almacenar el año de lanzamiento.
grouped['year'] = grouped['release_date'].dt.year

# Ahora, agrupa los datos por 'year' y 'genres', y suma las horas jugadas ('playtime_forever') para cada combinación.
resultados = grouped.groupby(['year', 'genres'])['playtime_forever'].sum().reset_index()

# Finalmente, puedes iterar sobre los resultados para mostrar la cantidad de horas por género y año.
for index, row in resultados.iterrows():
    print(f"Año {row['year']} género {row['genres']} playtime_forever: {row['playtime_forever']}")

Año 1983 género Action playtime_forever: 3582
Año 1983 género Adventure playtime_forever: 3582
Año 1983 género Casual playtime_forever: 3582
Año 1984 género Action playtime_forever: 384
Año 1984 género Adventure playtime_forever: 384
Año 1984 género Casual playtime_forever: 384
Año 1987 género Adventure playtime_forever: 7716
Año 1987 género Simulation playtime_forever: 235
Año 1988 género Action playtime_forever: 16243
Año 1988 género Adventure playtime_forever: 30241
Año 1988 género Indie playtime_forever: 13998
Año 1988 género RPG playtime_forever: 13998
Año 1988 género Simulation playtime_forever: 1
Año 1988 género Strategy playtime_forever: 13998
Año 1989 género Action playtime_forever: 607
Año 1989 género Adventure playtime_forever: 9324
Año 1989 género Simulation playtime_forever: 5084
Año 1990 género Action playtime_forever: 18787
Año 1990 género Adventure playtime_forever: 5538
Año 1990 género Simulation playtime_forever: 1926
Año 1990 género Strategy playtime_forever: 60
Año 

In [77]:
resultados['genres'] = resultados['genres'].replace("Design &amp; Illustration", "Design and Illustration", regex=True)

In [10]:
df ['genres'] = df['genres'].replace("animation &amp; modeling", "animation and modeling", regex=True)

In [82]:
resultados.to_csv('funcionuno.csv', index=False)

In [1]:
def PlayTimeGenre(genero: str):
    # Carga el archivo CSV en un DataFrame
    df = pd.read_csv('funcionuno.csv')  # Reemplaza 'tu_archivo.csv' con la ruta correcta de tu archivo

    # Convierte todas las filas de la columna 'genres' a minúsculas
    df['genre'] = df['genre'].str.lower()

    # Convierte el valor de 'genero' a minúsculas
    genero = genero.lower()

    # Filtra las filas del DataFrame por género
    genero_df = df[df['genre'] == genero]

    if genero_df.empty:
        return {"Año con más horas jugadas para Género": "No se encontraron datos para este género"}

    # Encuentra el año con más horas jugadas
    año_con_mas_horas = genero_df.loc[genero_df['hours_played'].idxmax()]['year']

    return {"Año con más horas jugadas para Género": año_con_mas_horas}

In [8]:
    # Carga el archivo CSV en un DataFrame
df = pd.read_csv('funcionuno.csv')  # Reemplaza 'tu_archivo.csv' con la ruta correcta de tu archivo

    # Convierte todas las filas de la columna 'genres' a minúscul
df['genres'] = df['genres'].str.lower()


In [11]:
df.to_csv("funcionuno.csv",index=False)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   year              340 non-null    int64 
 1   genres            340 non-null    object
 2   playtime_forever  340 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 8.1+ KB


In [16]:
def PlayTimeGenre(genero: str):
    # Carga el archivo CSV en un DataFrame
    df = pd.read_csv('funcionuno.csv', dtype={'genres': str})  # Reemplaza 'tu_archivo.csv' con la ruta correcta de tu archivo

    # Convierte el valor de 'genero' a minúsculas
    genero = genero.lower()

    # Filtra las filas del DataFrame por género
    genero_df = df[df['genres'] == genero]

    if genero_df.empty:
        return {f"Año con más horas jugadas para {genero}": "No se encontraron datos para este género"}

    # Encuentra el año con más horas jugadas
    año_con_mas_horas = genero_df.loc[genero_df['playtime_forever'].idxmax()]['year']

    return {f"Año con más horas jugadas para {genero}": año_con_mas_horas}

In [29]:
def PlayTimeGenre1(genero: str):
    # Carga el archivo CSV en un DataFrame
    df = pd.read_csv('funcionuno.csv', dtype={'genres': str})  # Reemplaza 'tu_archivo.csv' y 'nombre_columna' con los valores correctos

    # Convierte la columna a tipo int, ignorando errores
    df['playtime_forever'] = pd.to_numeric(df['playtime_forever'], errors='coerce')

    # Filtra las filas del DataFrame por género y asegúrate de que el género esté en minúsculas
    genero_df = df[df['genres'].str.lower() == genero.lower()]

    if genero_df.empty:
        return {f"Año con más horas jugadas para {genero}": "No se encontraron datos para este género"}

    año_con_mas_horas = genero_df.loc[genero_df['playtime_forever'].idxmax()]['year']
    return {f"Año con más horas jugadas para {genero}": año_con_mas_horas}

In [30]:
PlayTimeGenre1('Indie')

{'Año con más horas jugadas para Indie': 2006}

In [21]:
df = pd.read_csv('funcionuno.csv')

In [24]:
df['playtime_forever'] = df['playtime_forever'].astype(int)
df['year'] = df['year'].astype(int)
df['genres'] = df['genres'].astype(str)








In [26]:
df

,year,genres,playtime_forever
0,1983,action,3582
1,1983,adventure,3582
2,1983,casual,3582
3,1984,action,384
4,1984,adventure,384
...,...,...,...
335,2018,adventure,21
336,2018,early access,6128
337,2018,indie,2327
338,2018,rpg,3822


In [28]:
# Guardamos el DataFrame modificado en el mismo archivo CSV
df.to_csv('funcionuno.csv', index=False)
